(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Recommender System for Amazon Electronics

In this assignment, we will be working with the [Amazon dataset](http://cs-people.bu.edu/kzhao/teaching/amazon_reviews_Electronics.tar.gz). You will build a recommender system to make predictions related to reviews of Electronics products on Amazon.

Your grades will be determined by your performance on the predictive tasks as well as a brief written report about the approaches you took.

This assignment should be completed **individually**.

## Files

**train.json** 1,000,000 reviews to be used for training. It is not necessary to use all reviews for training if doing so proves too computationally intensive. The fields in this file are:

* **reviewerID** The ID of the reviewer. This is a hashed user identifier from Amazon.

* **asin** The ID of the item. This is a hashed product identifier from Amazon.

* **overall** The rating of reviewer gave the item.

* **helpful** The helpfulness votes for the review. This has 2 subfields, 'nHelpful' and 'outOf'. The latter is the total number of votes this review received. The former is the number of those that considered the review to be helpful.

* **reviewText** The text of the review.

* **summary** The summary of the review.

* **unixReviewTime** The time of the review in seconds since 1970.

**meta.json** Contains metadata of the items:

* **asin** The ID of the item.

* **categories** The category labels of the item being reviewed.

* **price** The price of the item.

* **brand** The brand of the item.

**pairs_Rating.txt** The pairs (reviewerID and asin) on which you are to predict ratings.

**pairs_Purchase.txt** The pairs on which you are to predict whether a user purchased an item or not.

**pairs_Helpful.txt** The pairs on which you are to predict helpfulness votes. A third column in this file is the total number of votes from which you should predict how many were helpful.

**helpful.json** The review data associated with the helpfulness prediction test set. The 'nHelpful' field has been removed from this data since that is the value you need to predict above. This data will only be of use for the helpfulness prediction task.

**baseline.py** A simple baseline for each task.

## Tasks

**Rating prediction** Predict people's star ratings as accurately as possible for those (reviewerID, asin) pairs in 'pairs_Rating.txt'. Accuracy will be measured in terms of the [root mean-squared error (RMSE)](http://www.kaggle.com/wiki/RootMeanSquaredError).

**Purchase prediction** Predict given a (reviewerID, asin) pair from 'pairs_Purchase.txt' whether the user purchased the item (really, whether it was one of the items they reviewed). Accuracy will be measured in terms of the [categorization accuracy](http://www.kaggle.com/wiki/HammingLoss) (1 minus the Hamming loss).

**Helpfulness prediction** Predic whether a user's review of an item will be considered helpful. The file 'pairs_Helpful.txt' contains (reviewerID, asin) pairs with a third column containing the number of votes the user's review of the item received. You must predict how many of them were helpful. Accuracy will be measured in terms of the total [absolute error](http://www.kaggle.com/wiki/AbsoluteError), i.e. you are penalized one according to the difference |nHelpful - prediction|, where 'nHelpful' is the number of helpful votes the review actually received, and 'prediction' is your prediction of this quantity.

We set up competitions on Kaggle to keep track of your results compared to those of other members of the class. The leaderboard will show your results on half of the test data, but your ultimate score will depend on your predictions across the whole dataset.
* Kaggle competition: [rating prediction](https://inclass.kaggle.com/c/cs591-hw3-rating-prediction3) click here to [join](https://kaggle.com/join/datascience16rating)
* Kaggle competition: [purchase prediction](https://inclass.kaggle.com/c/cs591-hw3-purchase-prediction) click here to [join](https://kaggle.com/join/datascience16purchase)
* Kaggle competition: [helpfulness prediction](https://inclass.kaggle.com/c/cs591-hw3-helpful-prediction) click here to [join](https://kaggle.com/join/datascience16helpful)

In [1]:
import requests
import json
import numpy as np
from collections import defaultdict
from scipy import linalg
import statsmodels.api as sm
import matplotlib.pyplot as plt

def readJson(f):
    for l in open(f):
        yield eval(l)

# Task 1

In [ ]:
predictions = open("predictions_Rating.txt", 'w')

itemRatings = defaultdict(dict)
for l in readJson('amazon_reviews_Electronics/train.json'):
    userId,itemId,rating = l['reviewerID'],l['asin'],l['overall']
    itemRatings[itemId][userId] = rating
    
allRatings = []
userRatings = defaultdict(dict)
for l in readJson('amazon_reviews_Electronics/train.json'):
    userId,itemId,rating = l['reviewerID'],l['asin'],l['overall']
    allRatings.append(rating)
    userRatings[userId][itemId] = rating  
    
globalAverage = (sum(allRatings) / len(allRatings)) 

userAverage = {}
for u in userRatings.keys():
    userAverage[u] = (sum(userRatings[u].values()) / len(userRatings[u]))
    
itemAverage = {}
for i in itemRatings.keys():
    if len(itemRatings[i]) == 0:
        itemAverage[i] = globalAverage
    else:
        itemAverage[i] = (sum(itemRatings[i].values()) / len(itemRatings[i]))

    
userdensity = {}
for user in userRatings.keys():
    userdensity[user] = len(userRatings[user].keys())
itemdensity = {}
for item in itemRatings.keys():
    itemdensity[item] = len(itemRatings[item].keys())

for l in open("amazon_reviews_Electronics/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    #play with the biases
    if i in itemAverage:
        this_item_average = itemAverage[i]
    else:
        this_item_average = globalAverage
    if u in userAverage:
        this_user_average = userAverage[u]
    else:
        this_user_average = globalAverage
    this_item_bias = this_item_average - globalAverage
    this_user_bias = this_user_average - globalAverage
    
    dodal = {}#takes into account all others that rated i, could add it to value as another factor
    dod = []
    b =0
    for item_user in itemRatings[i].keys():
        total_score = [0.0,0.0]
        common_items = list(set(userRatings[u].keys()).intersection(userRatings[item_user].keys()))
        if common_items != []: 
            for common_item in common_items:
                score = float(itemRatings[common_item][u])-float(itemRatings[common_item][item_user])
                total_score[0] = total_score[0] + score 
                total_score[1] = total_score[1] + 1
                valueq = total_score[0]/total_score[1]
                dod.append(valueq)
            #valueq = total_score[0]/total_score[1]
            #dodal[item_user] = userRatings[item_user][i]+valueq
        else: #no common items
            dod.append(userRatings[item_user][i])
            #dodal[item_user] = userRatings[item_user][i]
        b = b + 1
    if len(dod) == 0.0:
    #if len(dodal) == 0.0: #no one has reviewed item
        val= this_user_average
    else:
        val = sum(dod)/len(dod)
        #val = sum(dodal.values())/len(dodal.keys())  

    if u in userdensity and i in itemdensity:
        unum=userdensity[u]
        inum=itemdensity[i]
        #change
        total = float(unum +inum)
        #here
        value = (unum/total)*this_user_average+(inum/total)*this_item_average
        #print (value)
        if value>5:
            value = 5
        if value<0:
            value = 0
        predictions.write(u + '-' + i + ',' + str(value*0.68+0.32*globalAverage) + '\n')
    elif i in itemRatings:
        #here
        predictions.write(u + '-' + i + ',' + str((this_item_average)*0.68+0.32*globalAverage) + '\n')
    elif u in userRatings:
        #and here
        predictions.write(u + '-' + i + ',' + str((this_user_average)*0.68+0.32*globalAverage) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')
predictions.close()

# Task 2

In [110]:
itemCount = defaultdict(int)
userCount = defaultdict(int)
totalPurchases = 0
user = defaultdict(list)
itms = defaultdict(list)
for l in readJson('amazon_reviews_Electronics/train.json'):
    item = l['asin']
    us = l['reviewerID']
    user[us].append(item)
    itms[item].append(us)
    itemCount[item] += 1
    userCount[us] += 1
    totalPurchases += 1

mostPopular = [(itemCount[x], x) for x in itemCount]
mostPopular = sorted(mostPopular, key=lambda l:l[0], reverse=True)
#print(mostPopular[:10])
mostPopularU = [(userCount[x], x) for x in userCount]
mostPopularU = sorted(mostPopularU, key=lambda l:l[0], reverse=True)

rateCount=defaultdict(int)
for i in itemCount:
    rateCount[i]=float(itemCount[i])/float(totalPurchases)

[(1961, '5fb4315f07cee12b0ce46334502b2a86d2daf9a6f6dcd640ea3a536559acfb8b'), (1469, '949939abbb7f1e3ac2b96a28aec87f7c9a69cb09e245cb55564ab19b351a0de2'), (1454, 'c69cf5219ae5bb27222337bc48e2b5e817beb600f56f9c0737ac49956b830c55'), (1230, '8b4e1f1147ca3d2543cf858fa938e02dc8bcdae2cfaf210764b9619900559369'), (1162, '83e320747e56f950ac84674f3efafc8cfcbc5ac03e8c724b48d28d0e20b825f1'), (1020, '122523fc4c45d12460951c05009823e09da946eaf74b33e4a3c24cfd20a22304'), (913, 'ea8882af7dd64b29968dc5855be9f5d52795e511416363c0eed4185020c86212'), (834, '9e0be75a56380864463a8eacc100052df02fd484e6e299d5a86d32ae91c72aab'), (819, '214beb5aea52e92fed323297958570e2b7bbb58a436c8695761692cbfb3f3601'), (749, 'ee7459063a0309cd28c2cc159c0e8826334194674633a70dd5f12d3c95f2a790')]


In [122]:
print(len(itms))
print(len(user))
#print(float(len(user))/float(len(itms)))

uu = defaultdict(int)
for us in user:
    uu[us]=len(user[us])
print('items per user')
print(float(sum(uu.values()))/len(uu.keys()))    
    
ii = defaultdict(int)
for it in itms:
    ii[it]=len(itms[it])
print('users per item')    
print(float(sum(ii.values()))/len(ii.keys()))

173888
509678
items per user
1.96202308124
users per item
5.75082811925


In [100]:
icat = defaultdict(list)
categories = defaultdict(list)
k=0
for l in readJson('amazon_reviews_Electronics/meta.json'):
    k=k+1
    item = l['asin']
    cat = l['categories']
    if cat.startswith("[['Electronics', '"):
        cat= cat[18:]
        cat= cat[:len(cat)-3]
        if '], [' in cat:
            cat = cat.replace('], [', ', ')
            cat= cat.split("', '")
            #print(cat)
        else:
            cat= cat.split("', '")
            #print(cat)
        for c in cat:
                categories[c].append(item)
    if item not in icat:
        icat[item]=l['categories']
        if icat[item].startswith("[['Electronics', '"):
            icat[item]= icat[item][18:]
            icat[item]= icat[item][:len(icat[item])-3]
            if '], [' in icat[item]:
                icat[item] = icat[item].replace('], [', ', ')
                icat[item] = icat[item].split("', '")
                if 'Electronics' in icat[item]:
                    icat[item].remove('Electronics')
            else:
                icat[item] = icat[item].split("', '")
                if 'Electronics' in icat[item]:
                    icat[item].remove('Electronics')
        else:
            icat[item]= icat[item][3:]
            icat[item]= icat[item][:len(icat[item])-3]
            if '], [' in icat[item]:
                icat[item] = icat[item].replace('], [', ', ')
                icat[item] = icat[item].split("', '")
                if 'Electronics' in icat[item]:
                    icat[item].remove('Electronics')
            else:
                icat[item] = icat[item].split("', '")
                if 'Electronics' in icat[item]:
                    icat[item].remove('Electronics')

ucat = defaultdict(dict)             
for u in user:
    for i in user[u]:
        if i in icat:
            for c in icat[i]:
                if c in ucat[u]:
                    ucat[u][c]=ucat[u][c]+1
                else:
                    ucat[u][c]=1
                    
            
#need to do: 1get for each user the amount of times he purchases each category
#            2get for each category, how many times was purchased in total
#            3then take the categories of item in question.
#            4get how many times on average a user has used a category. 2/numusersintrain
#            5if amount of user>average then 1

In [109]:
cat ={}
for item in itemCount:
    for c in icat[item]:
        if c in cat:
            cat[c]=cat[c]+itemCount[item]
        else:
            cat[c]=itemCount[item]
            
catav = defaultdict(int)
for c in cat:
    catav[c]=float(cat[c])/1000000.0
    

In [ ]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/1.6: break

return2 = set()
count = 0
for ic, i in mostPopularU:
    count += ic
    return2.add(i)
    if count > totalPurchases/125: break
        
predictions = open("predictions_Purchase.txt", 'w')
k=0
n=0
o=0
for l in open("amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    numb=0
    if i in return1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

print(k)  
print(n)
print(o)                      
predictions.close()

In [129]:
# Do not run, failed attempt
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/1.6: break

return2 = set()
count = 0
for ic, i in mostPopularU:
    count += ic
    return2.add(i)
    if count > totalPurchases/125: break
        
predictions = open("predictions_Purchase.txt", 'w')
k=0
n=0
o=0
for l in open("amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    numb=0
    if u in uu and i in ii:
        if uu[u]<=2 and ii[i]>=6:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
    else:
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")

print(k)  
print(n)
print(o)                      
predictions.close()

0
0
0


In [112]:
# Do not run, failed attempt
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/1.6: break

return2 = set()
count = 0
for ic, i in mostPopularU:
    count += ic
    return2.add(i)
    if count > totalPurchases/125: break
        
predictions = open("predictions_Purchase.txt", 'w')
k=0
n=0
o=0
for l in open("amazon_reviews_Electronics/pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    num_of_common_cat=0
    if i in icat and u in ucat:
        this_item_cat = icat[i]
        this_user_cat = ucat[u]
        omega=0
        for usrs in itms[i]:
            for c in ucat[usrs]:
                if c in ucat[u]:
                    omega = omega+1
                else:
                    omega= omega-1
        if omega >= 0:
            predictions.write(u + '-' + i + ",1\n")
            n=n+1
        else:
            predictions.write(u + '-' + i + ",0\n")
            k=k+1
    else:
        o=o+1
        if i in return1:
            predictions.write(u + '-' + i + ",1\n")
        elif u in return2:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")

print(k)  
print(n)
print(o)                      
predictions.close()

40175
33808
26017


# Task 3

In [22]:
allHelpful = []
userHelpful = defaultdict(list)
itemHelpful = defaultdict(list)
userHelpfuld = defaultdict(dict)
itemHelpfuld = defaultdict(dict)
textu = defaultdict(dict)
texti = defaultdict(dict)
score = defaultdict(dict)
userOutOfs = defaultdict(dict)

for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item = l['reviewerID'],l['asin']
    allHelpful.append(l['helpful'])
    textu[user][item] = len(l['reviewText'])
    score[user][item] = l["overall"]
    a = l["helpful"]
    userOutOfs[user][item] = a['outOf']
    userHelpfuld[user][item] = float(a['nHelpful'])/float(a['outOf'])


a=[]    
rt=0
mama = defaultdict(int)
mamnum = defaultdict(int)

tlen = defaultdict(dict)
rt = 0
for u in userHelpfuld:
    for i in userHelpfuld[u]:
        mama[str(userOutOfs[u][i])] = mama[str(userOutOfs[u][i])] + userHelpfuld[u][i]
        mamnum[str(userOutOfs[u][i])] = mamnum[str(userOutOfs[u][i])]+1
        if userOutOfs[u][i]==2 and textu[u][i]<100:
            rt = rt+1
            tlen[u][i]=userHelpfuld[u][i]
            
print(rt)
tlensum=0
for gh in tlen:
    tlensum=tlensum+sum(tlen[gh].values())
print(tlensum)
print(tlensum/rt)            
lis=defaultdict(dict)            
for mam in mama:
    lis[mam]= mama[mam]/mamnum[mam]




text = defaultdict(dict)
userOutOf = defaultdict(dict)
for l in readJson("amazon_reviews_Electronics/helpful.json"):
    use,ite = l['reviewerID'],l['asin']
    userOutOf[use][ite] = l['outOf']
    text[use][ite] = len(l['reviewText'])
    
predictions = open("predictions_Helpful.txt", 'w')
for l in open("amazon_reviews_Electronics/pairs_Helpful.txt"):
    boo = False 
    if l.startswith("reviewerID"):
    #header
        predictions.write(l)
        continue
    u,i,outOf = l.strip().split('-')
    outOf = int(outOf)
    o = outOf
    while boo == False:
        if o==1:
            boo = True
            predictions.write(u + '-' + i + '-' + str(o) + ',' + str(1) + '\n')
        elif str(outOf) in lis:
            boo = True
            predictions.write(u + '-' + i + '-' + str(o) + ',' + str(outOf*lis[str(outOf)]) + '\n')
        outOf = outOf-1    
predictions.close()


3583
1905.5
0.531816913201


## Grading and Evaluation

You will be graded on the following aspects.

* Your written report. This should describe the approaches you took to each of the 3 tasks. To obtain good performance, you should not need to invent new approaches (though you are more than welcome to) but rather you will be graded based on your decision to apply reasonable approaches to each of the given tasks. (**10pts** for each task)

* Your ability to obtain a solution which outperforms the baselines on the unseen portion of the test data. Obtaining full marks requires a solution which is substantially better (at least several percent) than baseline performance. (**10pts** for each task)

* Your ranking for each of the three tasks compared to other students in the class. (**5pts** for each task)

* Obtain a solution which outperforms the baselines on the seen portion of the test data (the leaderboard). 
(**5pts** for each task)

## Baselines

Simple baselines have been provided for each of the 3 tasks. These are included in 'baselines.py' among the files above. These 3 baselines operate as follows:

**Rating prediction** Returns the global average rating, or the user's average if you have seen them before in the training data.

**Purchase prediction** Finds the most popular products that account for 50% of purchases in the training data. Return '1' whenever such a product is seen at test time, '0' otherwise.

** Helpfulness prediction** Multiplies the number of votes by the global average helpfulness rate, or the user's rate if we saw this user in the training data.

Running 'baseline.py' produces 3 files containing predicted outputs. Your submission files should have the same format.

## Dataset Citation

**Image-based recommendations on styles and substitutes** J. McAuley, C. Targett, J. Shi, A. van den Hengel *SIGIR*, 2015

**Inferring networks of substitutable and complementary products** J. McAuley, R. Pandey, J. Leskovec *Knowledge Discovery and Data Mining*, 2015

-----------------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()